# Overview

This Python script queries the ContentDM API and downloads metadata Temple University Libraries' Digital Collections. 

# Install Packages


In [ ]:
from google.colab import files
%load_ext google.colab.data_table
import requests, csv
import json
import pandas as pd
from pandas.io.json import json_normalize
import cv2
import os
import urllib.request
import urllib3
from urllib3 import request

# Query ContentDM

## Download Temple Metadata for Digital collections

In [ ]:
manifestUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/manifest.json'

In [ ]:
resp = urllib.request.urlopen(manifestUrl)
data = resp.read().decode("utf-8")
data = json.loads(data)
data

In [ ]:
df = pd.json_normalize(data['collections'])
del df['@type']
df.rename(columns = {'@id':'ID', 'label':'Label'}, inplace = True)
df.head()

,ID,Label
0,https://cdm16002.contentdm.oclc.org/iiif/info/...,Allied Posters of World War I
1,https://cdm16002.contentdm.oclc.org/iiif/info/...,Ambler Campus History in Photographs
2,https://cdm16002.contentdm.oclc.org/iiif/info/...,City Parks Association Photographs
3,https://cdm16002.contentdm.oclc.org/iiif/info/...,Frank G. Zahn Railroad Photograph Collection
4,https://cdm16002.contentdm.oclc.org/iiif/info/...,Franklin H. Littell Papers


In [ ]:
df.to_csv (r'manifest.csv', index = False, header=True)

In [ ]:
files.download('manifest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Query a Collection

The manifestURL variable determines what collection, such as WWII Posters, are queried. 

In [ ]:
collectionUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll9/p1.json'

In [ ]:
responseManifest = requests.get(collectionUrl)
m = json.loads(responseManifest.text)
m

In [ ]:
csv_out = csv.writer(open('collection.csv', 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
csv_out.writerow(['Title', 'Date', 'Contributor', 'Description', 'Subject', 'ID'])

47

In [ ]:
for i in m['manifests']:
    title = date = contributor = description = subject = ID = ''
    
    manifestItemUrl = i['@id']
    responseManifestItem = requests.get(manifestItemUrl)
    manifestItem = json.loads(responseManifestItem.text)

    title = manifestItem['metadata'][0]['value']
    date = manifestItem['metadata'][1]['value']
    contributor = manifestItem['metadata'][2]['value']
    description = manifestItem['metadata'][3]['value']
    subject = manifestItem['metadata'][4]['value']
    ID = i['@id']
    csv_out.writerow([title, date, contributor, description, subject, ID])    

In [ ]:
files.download('collection.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = pd.read_csv('collection.csv')
df.head()

,Title,Date,Contributor,Description,Subject,ID
0,Sottoscrivete al prestito,[ca. 1917],"Capranesi, Giovanni, 1852-1921, artist.; Istit...",Allegorical depiction of Italy as a female wea...,"Date of publication from Rawls, Walton, Wake u...",https://cdm16002.contentdm.oclc.org/iiif/info/...
1,Step into your place,Poster (Great Britain. Parliamentary Recruitin...,1915,Great Britain. Parliamentary Recruiting Commit...,British recruiting poster. Illustration depic...,https://cdm16002.contentdm.oclc.org/iiif/info/...
2,Indian Ink,1916,"Pereira, Harold B., artist.;",Poster advertising the Christmas 1916 edition ...,"Signed: Harold B. Pereira, 16 [i.e. 1916?]. Or...",https://cdm16002.contentdm.oclc.org/iiif/info/...
3,Per la libertà e la civiltà del mondo,1918,"Dudovich, Marcello, 1878-1962, artist.; Atelie...",Italian poster urging subscription to the nati...,Signed: M. Dudovich. Date of publication from ...,https://cdm16002.contentdm.oclc.org/iiif/info/...
4,The watchers of the seas,[ca. 1914-1918],"Raven-Hill, L. (Leonard), 1867-1942, artist.; ...",British naval recruiting poster that asks for ...,"Signed: ""L. Ravenhill."" Date based on the date...",https://cdm16002.contentdm.oclc.org/iiif/info/...


# Image Download

This url is useful
manifestUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll4/4/manifest.json'